## A model build using TweetBERT to classify tweet as causal or non-causal 

In [1]:
import pandas as pd
import numpy as np
import spacy 
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import random
import os
import torch.nn.functional as F
import torch
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import transformers
from tqdm import tqdm, trange
#from google.colab import drive, files
import io

In [2]:
#uploaded = files.upload()
#data = pd.read_excel(io.BytesIO(uploaded['Causality + hypoglycemia.xlsx']), sheet_name=">5000_samples_")


# data = pd.read_excel("/home/adrian/Downloads/Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")
data = pd.read_excel("Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")

<ipython-input-2-19ae4d9dded0>:6: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  data = pd.read_excel("Causality + hypoglycemia.xlsx", sheet_name=">5000_samples_")


In [3]:
print("Total count:", data.shape[0])
data = data[data["Causal association"].notnull()]
print("Labeled count:", data.shape[0])

data.head()

Total count: 5456
Labeled count: 5000


,id,text,full_text,Intent,Cause,Effect,Causal association,Charline association0=no;1=yes,Remarks
0,908171203029868545,"tonight , I learned my older girl will back he...","tonight , I learned my older girl will back he...",NaN,NaN,NaN,0.0,NaN,NaN
1,1203645589214367745,USER USER I knew diabetes and fibromyalgia wer...,USER USER I knew diabetes and fibromyalgia wer...,joke,NaN,NaN,0.0,NaN,NaN
2,1310596731063525376,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,mS,NaN,NaN,0.0,NaN,NaN
3,1125198453167022085,USER Cheers ! Have one for this diabetic too !,USER Cheers ! Have one for this diabetic too !,mS,NaN,NaN,0.0,NaN,NaN
4,1248600944138268673,USER Additionally the medicines are being char...,USER Additionally the medicines are being char...,NaN,medicines are being charged at MRP,costing much higher,1.0,NaN,NaN


### Interrater-reliabilty measure

In [4]:
from sklearn.metrics import cohen_kappa_score

charline = data[data["Charline association0=no;1=yes"].notnull()]
coder1 = charline["Causal association"].values
coder2 = charline["Charline association0=no;1=yes"]
score = cohen_kappa_score(coder1,coder2)
print('Cohen\'s Kappa:',score)

Cohen's Kappa: 0.823719518166683


### Preprocessing

In [5]:
data["Causal association"].value_counts()

0.0    3720
1.0    1280
Name: Causal association, dtype: int64

In [6]:
# Normalisation for BertTweet
from nltk.tokenize import TweetTokenizer
from emoji import demojize
import re

tokenizer = TweetTokenizer()

# https://huggingface.co/vinai/bertweet-base
def normalizeToken(token):
    lowercased_token = token.lower()
    if token.startswith("@"):
        return "@USER"
    elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
        return "HTTPURL"
    elif len(token) == 1:
        return demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

def normalizeTweet(tweet):

    tokens = tokenizer.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll ", " 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.") .replace(" p . m ", " p.m ").replace(" a . m .", " a.m.").replace(" a . m ", " a.m ")

    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    
    return " ".join(normTweet.split())

In [7]:
def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    sentences = [s  for s in sentences if s != ""]
    return sentences


def create_training_data(data):
    tweets = []
    causal_labels = []
    
    for i, row in data.iterrows():
#        print("\n")
#        print(row["full_text"])
#        print(row["Intent"], "->", row["Causal association"])
       
        sentences = split_into_sentences(row["full_text"])
        intents = set(str(row["Intent"]).strip().split(";"))
        causes = str(row["Cause"]).strip().split(";")
        effects = str(row["Effect"]).strip().split(";")
#        print("\tintents:", intents)
#        print("\tcauses: '{}'".format(causes))
#        print("\teffects: '{}'".format(effects))
        
        if set({"nan"}) == intents or set({" "}) == intents: # single sentence
#            print("\tA: add => causality: {}".format(row["Causal association"]))
            tweets.append(row["full_text"])
            causal_labels.append(row["Causal association"])

        elif (
             set({"q"}) == intents 
          or set({"joke"}) == intents 
          or set({"q", "joke"}) == intents
          or set({"joke", "mS"}) == intents 
          or set({"neg"}) == intents 
          or set({"neg", "msS"}) == intents
          or set({"neg", "mS"}) == intents
          or set({"neg", "msS", "mE"}) == intents
          or set({"q", "joke", "mS"}) == intents
          or set({"q", "msS", "neg"}) == intents
          or set({"neg", "mC"}) == intents
          or set({"mC", "joke", "msS"}) == intents
          or set({"joke", "mE"}) == intents
        ):
#            print("\tB ignore")
            continue
        elif (  
             set({"mS"}) == intents # multiple sentences (possible that cause and effect in different sentences -> ignore)
          or set({"q", "mS"}) == intents # multiple sentences or question
          or set({"mS", "mE"}) == intents
          or set({"mC", "mS"}) == intents
          or set({"mC", "mS", "mE"}) == intents
          or set({"q", "mC", "mS"}) == intents
          or set({"q", "mC", "mS", "mE"}) == intents
            
        ):
            for sent in sentences:
#                print(sent)
                if sent[-1] != "?": # ignore questions
#                    print("\tC: add, causality => 0")
                    tweets.append(sent)
                    causal_labels.append(0)  
#                else:
#                    print("\tD: ignore")
        elif (
            set({"msS"}) == intents # multiple sentences with cause and effect in single sentence
         or set({"q", "msS"}) == intents # msS and a question
         or set({"msS", "mE"}) == intents # msS with several effects
         or set({"mC", "msS"}) == intents
         or set({"mE"}) == intents # multiple effects
         or set({"mC"}) == intents # multiple causes
         or set({"mC", "msS", "mE"}) == intents
         or set({"mC", "mE"}) == intents
         or set({"q", "mC", "mE"}) == intents   
         or set({"q", "mC", "msS"}) == intents
        ):

            if row["Causal association"] != 1: #TEST
                print(sentences)
                print("1) ERROR: Causal association should be 1 !!!!")      
                print()
        
            for sent in sentences:
#                print("sub sent:", sent)
                if sent[-1] != "?": # ignore question
                    
                    existCause = False
                    for cause in causes:
                        if cause in sent:
#                            print("Cause: <{}> in sent".format(cause))
                            existCause = True
                    
                    existEffect = False
                    for effect in effects:
                        if effect in sent:
#                            print("Effect: <{}> in sent".format(effect))
                            existEffect = True
                            
                    if existCause and existEffect:
                        tweets.append(sent)
                        causal_labels.append(row["Causal association"])
#                        print("E: add with Cause + effect => association: {}".format(row["Causal association"]))
                    else:
                        tweets.append(sent)
                        causal_labels.append(0)
#                        print("F: not both C + E in sentence, association: 0")
#                else:
#                    print("H: question in sentence")
            if row["Causal association"] == 0:
                print(sentences)
                print("H: should not have causality == 0")
#        else:
#            print("END: should not happen")

    
    return pd.DataFrame({"tweet" : tweets, "Causal association" : causal_labels})

trainingData = create_training_data(data)
print("N sentences:", trainingData.shape)
trainingData = trainingData[trainingData["tweet"].str.split(" ").str.len() > 3] # keep only sentence with more than 3 tokens
print("N sentences with > 3 words:", trainingData.shape)

N sentences: (9779, 2)
N sentences with > 3 words: (8235, 2)


In [8]:
# let's print a few example of our cleaned and normalized traingin dataset
trainingData.head()

,tweet,Causal association
0,"tonight , I learned my older girl will back he...",0.0
1,⬇ ️ ⬇ ️ ⬇ ️ THIS ⬇ ️ ⬇ ️ ⬇ ️ My wife has type ...,0.0
2,I'm a trans woman .,0.0
3,"Both of us could use a world where "" brave and...",0.0
4,"Make a world where people can just be , withou...",0.0


In [9]:
trainingData["Causal association"].value_counts()

0.0    7218
1.0    1017
Name: Causal association, dtype: int64

### Training

In [10]:
# text = trainingData["tweet"].map(normalizeTweet).values.tolist()
# labels = trainingData["Causal association"].values.tolist()
# # first split the data into traingin and testing label in the ratio of 80:20
# train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=0.2)
# # split the new training data (80% of actual data) to get train and validation set
# train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)



# print("Train: {}".format(len(train_texts)))
# print("Val: {}".format(len(val_texts)))
# print("Test: {}".format(len(test_texts)))


In [11]:
## when not straitifyign the data: 

text = trainingData["tweet"].map(normalizeTweet).values.tolist()
labels = trainingData["Causal association"].values.tolist()
# first split the data into training and testing label in the ratio of 80:20
train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=0.2)
# split the new training data (80% of actual data) to get train and validation set
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2)

data_count_info = pd.Series(labels).value_counts(normalize=True)
train_count_info = pd.Series(train_labels).value_counts(normalize=True)
val_count_info = pd.Series(val_labels).value_counts(normalize=True)
test_count_info = pd.Series(test_labels).value_counts(normalize=True)

# for class-imbalanced dataset, the class weight for a ith class
# to be specified for balancing in the loss function is given by:
# weight[i] = num_samples / (num_classes * num_samples[i])
# since train_count_info obtained above has fraction of samples
# for ith class, hence the corresponding weight calculation is:
class_weight = (1/train_count_info)/len(train_count_info)

print("All: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(labels), *data_count_info.round(4).to_list()))
print("Train: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(train_labels), *train_count_info.round(4).to_list()))
print("Val: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(val_labels), *val_count_info.round(4).to_list()))
print("Test: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(test_labels), *test_count_info.round(4).to_list()))
print("Balancing class wts: for 0 = {}, for 1 = {}".format(
    *class_weight.round(4).to_list()))

All: Count = 8235, % of 0 = 0.8765, % of 1 = 0.1235
Train: Count = 5270, % of 0 = 0.8818, % of 1 = 0.1182
Val: Count = 1318, % of 0 = 0.871, % of 1 = 0.129
Test: Count = 1647, % of 0 = 0.864, % of 1 = 0.136
Balancing class wts: for 0 = 0.567, for 1 = 4.2295


In [12]:
####################### Stratified splits ####################

text = trainingData["tweet"].map(normalizeTweet).values.tolist()
labels = trainingData["Causal association"].values.tolist()
# first split the data into training and testing label in the ratio of 80:20
train_texts, test_texts, train_labels, test_labels = train_test_split(text, labels, test_size=0.2, stratify=labels)
# split the new training data (80% of actual data) to get train and validation set
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, stratify=train_labels)



data_count_info = pd.Series(labels).value_counts(normalize=True)
train_count_info = pd.Series(train_labels).value_counts(normalize=True)
val_count_info = pd.Series(val_labels).value_counts(normalize=True)
test_count_info = pd.Series(test_labels).value_counts(normalize=True)

# for class-imbalanced dataset, the class weight for a ith class
# to be specified for balancing in the loss function is given by:
# weight[i] = num_samples / (num_classes * num_samples[i])
# since train_count_info obtained above has fraction of samples
# for ith class, hence the corresponding weight calculation is:
class_weight = (1/train_count_info)/len(train_count_info)

print("All: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(labels), *data_count_info.round(4).to_list()))
print("Train: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(train_labels), *train_count_info.round(4).to_list()))
print("Val: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(val_labels), *val_count_info.round(4).to_list()))
print("Test: Count = {}, % of 0 = {}, % of 1 = {}".format(
    len(test_labels), *test_count_info.round(4).to_list()))
print("Balancing class wts: for 0 = {}, for 1 = {}".format(
    *class_weight.round(4).to_list()))

All: Count = 8235, % of 0 = 0.8765, % of 1 = 0.1235
Train: Count = 5270, % of 0 = 0.8765, % of 1 = 0.1235
Val: Count = 1318, % of 0 = 0.8763, % of 1 = 0.1237
Test: Count = 1647, % of 0 = 0.8767, % of 1 = 0.1233
Balancing class wts: for 0 = 0.5705, for 1 = 4.0476


In [13]:
# train_texts[1]

In [14]:
# Transform labels + encodings into Pytorch DataSet object (including __len__, __getitem__)
class TweetDataSet(torch.utils.data.Dataset):
    def __init__(self, text, labels, tokenizer):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.text, padding=True, truncation=True, return_token_type_ids=True)
        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"]
        return {
                "input_ids" : torch.tensor(ids[idx], dtype=torch.long)
              , "attention_mask" : torch.tensor(mask[idx], dtype=torch.long)
              , "token_type_ids" : torch.tensor(token_type_ids[idx], dtype=torch.long)
              , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
        }      

    def __len__(self):
        return len(self.labels)

    
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base")

train_dataset = TweetDataSet(train_texts, train_labels, tokenizer)
val_dataset = TweetDataSet(val_texts, val_labels, tokenizer)
test_dataset = TweetDataSet(test_texts, test_labels, tokenizer)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

# put data to batches (iterables)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


5270
1318
1647


In [15]:
# !jupyter nbextension enable --py widgetsnbextension

In [16]:
# # from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
# # from transformers import AutoModelForSequenceClassification
# from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef

# def compute_metrics(pred, labels):
#     #labels = pred.label_ids
#     #preds = pred.predictions.argmax(-1)
#     precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='binary')
#     acc = accuracy_score(labels, pred)
#     return {
#         'accuracy': acc,
#         'f1': f1,
#         'precision': precision,
#         'recall': recall
#     }



# class CausalityBERT(torch.nn.Module):
#     """ Model Bert"""
#     def __init__(self):
#         super(CausalityBERT, self).__init__()
#         self.num_labels = 2
#         self.bert = transformers.BertModel.from_pretrained("vinai/bertweet-base")
#         self.dropout = torch.nn.Dropout(0.3)
#         self.linear = torch.nn.Linear(768, self.num_labels)
#         # softmax layer missing ? -> Vivek
        
#     def forward(self, input_ids, attention_mask, token_type_ids):
#         _, output_1 = self.bert(input_ids, attention_mask = attention_mask, token_type_ids=token_type_ids, return_dict=False)
#         output_2 = self.dropout(output_1)
#         logits = self.linear(output_2)
#         return logits


# ## Model parameters
# batchsize_train = 16
# lr = 5e-5
# adam_eps = 1e-8
# epochs = 3 
# num_warmup_steps = 0
# num_training_steps = len(train_loader)*epochs

# # Store our loss and accuracy for plotting
# train_loss_set = []
# learning_rate = []


# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# #model = AutoModelForSequenceClassification.from_pretrained("vinai/bertweet-base")
# model = CausalityBERT()
# model.to(device)

# # fine-tune only the task-specific parameters -> Vivek? 
# for param in model.bert.parameters():
#     param.requires_grad = False
    
# model.to(device)
# model.train() # set model to training mode


# optim = AdamW(model.parameters(), lr=lr, eps=adam_eps)
# scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # scheduler with a linearly decreasing learning rate from the initial lr set in the optimizer to 0; after a warmup period durnig which it increases linearly from 0 to the initial lr set in the optimizer
# loss_fn = CrossEntropyLoss()

# for epoch in trange(1, epochs+1, desc='Epoch'):
#     print("<" + "="*22 + F" Epoch {epoch} "+ "="*22 + ">")
    
#     #calculate total loss for this epoch
#     batch_loss = 0
    
#     for batch in tqdm(train_loader):
#         #print("batch:", batch)
#         optim.zero_grad() # gradients get accumulated by default -> clear previous accumulated gradients
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         token_type_ids = batch["token_type_ids"].to(device)
#         labels = batch['labels'].to(device)
#         #logits = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) # forward pass
#         logits = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        
        
        
        
#         loss = loss_fn(logits, labels)
#         print("loss:", loss)
#         #loss = outputs[0]
#         loss.backward() # backward pass
#         optim.step()    # update parameters and take a step up using the computed gradient
#         scheduler.step()# update learning rate scheduler
#         batch_loss += loss.item() # update tracking variables
        
#     avg_train_loss = batch_loss / len(train_loader) # calculate avg loss over training data

#     # store the current learning rate
#     for param_group in optim.param_groups:
#         print("\n\tCurrent Learning rate: ", param_group['lr'])
#         learning_rate.append(param_group['lr'])
    
#     train_loss_set.append(avg_train_loss)
#     print(F'\n\tAverage Training loss: {avg_train_loss}')

    
#     ## ---- Validation ------
#     model.eval() # put model in evaluation mode for validation set
    
#     eval_accuracy, eval_mcc_accuracy, nb_eval_steps = 0, 0, 0 # Tracking variables
#     val_accuracy = []
#     val_loss = []
#     val_acc = []
#     val_prec = []
#     val_rec = []
#     val_f1 = []
    
    
#     # Evaluate data for one epoch
#     for batch in tqdm(validation_loader):
#         batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
#         b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
        
#         with torch.no_grad(): # tell model not to compute or store gradients -> saves memory + speeds up validation
#           #  logits = model(b_inputs_ids, attention_mask=b_input_mask, token_type_ids=b_token_type_ids) # forward pass, calculates logit predictions
#           logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions
            
#         loss = loss_fn(logits, b_labels)
#         val_loss.append(loss.item())
        
#         # move logits and labels to CPU
#         logits = logits.detach().to('cpu').numpy()
#         label_ids = b_labels.to('cpu').numpy()
        
#         pred_flat = np.argmax(logits, axis=1).flatten() # convert logits to list of predicted labels
#         labels_flat = label_ids.flatten()
#         eval_accuracy += accuracy_score(labels_flat, pred_flat)
#         eval_mcc_accrucay += matthews_corrcoef(labels_flat, pred_flat)        
#         metrics = compute_metrics(pred_flat, labels_flat)
#         val_acc.append(metrics["accuracy"])
#         val_prec.append(metrics["precision"])
#         val_rec.append(metrics["recall"])
#         val_f1.append(metrics["f1"])
#         nb_eval_steps += 1
        
#     print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
#     print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy / nb_eval_steps}')
#     print(F'\n\tValidation loss: {np.mean(val_loss)}')
#     print(F'\n\tValidation acc: {np.mean(val_acc)}')
#     print(F'\n\tValidation prec: {np.mean(val_prec)}')
#     print(F'\n\tValidation rec: {np.mean(val_rec)}')
#     print(F'\n\tValidation f1: {np.mean(val_f1)}')

    


In [17]:
# from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
# from transformers import AutoModelForSequenceClassification

## we are measuring weighted metrics - as our dataset is unbalanced 
# Calculate metrics for each label, and find their average weighted by support
#(the number of true instances for each label). 
#This alters ‘macro’ to account for label imbalance; 
# it can result in an F-score that is not between precision and recall.


from sklearn.metrics import accuracy_score, precision_recall_fscore_support, matthews_corrcoef

def compute_metrics(pred, labels):
    #labels = pred.label_ids
    #preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='weighted')
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



class CausalityBERT(torch.nn.Module):
    """ Model Bert"""
    def __init__(self):
        super(CausalityBERT, self).__init__()
        self.num_labels = 2
        self.bert = transformers.BertModel.from_pretrained("vinai/bertweet-base")
        self.dropout = torch.nn.Dropout(0.3)
        self.linear1 = torch.nn.Linear(768, 256)
        self.linear2 = torch.nn.Linear(256, self.num_labels)
        self.softmax = torch.nn.Softmax(-1)
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        _, output_1 = self.bert(input_ids, attention_mask = attention_mask, token_type_ids=token_type_ids, return_dict=False) # if output 1 is our cls token        
        output_2 = self.dropout(output_1)
        output_3 = self.linear1(output_2)  
        output_4 = self.dropout(output_3)
        output_5 = self.linear2(output_4)
        # cross entory will take care of the logits - we don't need if we are usign cross entropy for loss function 
        # if doing yourself - use nll loss and logSoftmax 
#         logit = self.softmax(output_5)
        return output_5



    


## Model parameters


In [18]:
batchsize_train = 16
lr = 5e-5
adam_eps = 1e-8
epochs = 35
num_warmup_steps = 0
num_training_steps = len(train_loader)*epochs

In [19]:


# Store our loss and accuracy for plotting : where is accuracy : it is loss an dlearning rate 
train_loss_set = []
learning_rate = []


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = CausalityBERT()
model.to(device)

# fine-tune only the task-specific parameters -> Vivek? 
## can we just update last layer of BERT 
for param in model.bert.parameters():
    param.requires_grad = False


optim = AdamW(model.parameters(), lr=lr, eps=adam_eps)
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps) # scheduler with a linearly decreasing learning rate from the initial lr set in the optimizer to 0; after a warmup period durnig which it increases linearly from 0 to the initial lr set in the optimizer
# loss_fn = CrossEntropyLoss()

## penalising more for class with less number of exaplmes 
loss_fn = CrossEntropyLoss(torch.tensor(class_weight.to_list()).to(device))


You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertModel: ['roberta.encoder.layer.0.output.dense.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.9.output.dense.weight', 'roberta.encoder.layer.5.attention.self.key.weight', 'roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.attention.self.key.weight', 'lm_head.bias', 'roberta.encoder.layer.6.attention.self.query.bias', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.4.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'robe

In [20]:



for epoch in trange(1, epochs+1, desc='Epoch'):
    print("<" + "="*22 + F" Epoch {epoch} "+ "="*22 + ">")
    
    #calculate total loss for this epoch
#     epoch_loss = 0
    
    ########### training eval metrics #############################
    tr_accuracy, tr_mcc_accuracy, nb_tr_steps = 0, 0, 0 # Tracking variables
    train_accuracy = []
    train_loss = []
    train_acc = []
    train_prec = []
    train_rec = []
    train_f1 = []
    
    ###################################################
    
    for batch in tqdm(train_loader):
        #print("batch:", batch)
        optim.zero_grad() # gradients get accumulated by default -> clear previous accumulated gradients
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch['labels'].to(device)
        #logits = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids) # forward pass
#         outputs = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        ###########################################################################
        model.train()
        ## this is output_5 based on our model defined above 
        logits = model(**{"input_ids":input_ids, "attention_mask":attention_mask, "token_type_ids":token_type_ids}) # forward pass
        #############################################################################
        loss = loss_fn(logits, labels)
        print("loss:", loss)
        #loss = outputs[0]
        loss.backward() # backward pass
        optim.step()    # update parameters and take a step up using the computed gradient
        scheduler.step()# update learning rate scheduler
#         epoch_loss += loss.item() # update tracking variables
        train_loss.append(loss.item())
        
    
    ############# Training Accuracy Measure ##########################################
#         loss = loss_fn(logits, b_labels)
#         val_loss.append(loss.item())

        # move logits and labels to CPU
        logits = logits.detach().to('cpu').numpy()
        label_ids = labels.to('cpu').numpy()

        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()

# #         tr_accuracy += accuracy_score(labels_flat, pred_flat) # this is same as metric accuracy 
#         tr_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  


        metrics = compute_metrics(pred_flat, labels_flat)
#         print(metrics)
        
        
#         train_loss.append(loss)
        train_acc.append(metrics["accuracy"])
        train_prec.append(metrics["precision"])
        train_rec.append(metrics["recall"])
        train_f1.append(metrics["f1"])
        
#         batch_loss
#         nb_tr_steps += 1
        
#     batch_loss = np.mean(train_loss)
#     train_loss_set.append(batch_loss)
        
    
#     print(F'\n\tTraining Accuracy: {tr_accuracy / nb_tr_steps}') # accuracy is calculated twice 
#     print(F'\n\tTraining MCC Accuracy: {tr_mcc_accuracy / nb_tr_steps}') # eval M
    
#     train_loss_set.append(batch_loss) # it should be either loss or batch_loss
    
    
#     train_loss = train_loss.to('cpu').numpy()
    print(F'\n\tTrain loss: {np.mean(train_loss)}')
    print(F'\n\ttrain acc: {np.mean(train_acc)}')
    print(F'\n\ttraining prec: {np.mean(train_prec)}')
    print(F'\n\ttraining rec: {np.mean(train_rec)}')
    print(F'\n\ttraining f1: {np.mean(train_f1)}')
    
    
    ###################################################################################
    #     avg_train_loss = batch_loss / len(train_loader) # calculate avg loss over training data : we don't need it as you did it for each sample
    #     train_loss_set.append(avg_train_loss)
    #     print(F'\n\tAverage Training loss: {avg_train_loss}')
    
#     train_loss_set.append(train_loss)
#     print(F'\n\tAverage Training loss per epoch: {train_loss}')


    # store the current learning rate
    for param_group in optim.param_groups:
        print("\n\tCurrent Learning rate: ", param_group['lr'])
        learning_rate.append(param_group['lr'])
    


    
    ## ---- Validation ------
#     model.eval() # put model in evaluation mode for validation set
    
    eval_accuracy, eval_mcc_accuracy, nb_eval_steps = 0, 0, 0 # Tracking variables
    val_accuracy = []
    val_loss = []
    val_acc = []
    val_prec = []
    val_rec = []
    val_f1 = []
    
    
    # Evaluate data for one epoch
    for batch in tqdm(validation_loader):
        batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
        b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
        
        with torch.no_grad(): # tell model not to compute or store gradients -> saves memory + speeds up validation
          #  logits = model(b_inputs_ids, attention_mask=b_input_mask, token_type_ids=b_token_type_ids) # forward pass, calculates logit predictions
#             outputs = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 
            ##################################################################################
            model.eval()
            logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 

                        
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())
        
        # move logits and labels to CPU
        logits = logits.detach().to('cpu').numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        
#         eval_accuracy += accuracy_score(labels_flat, pred_flat)
#         eval_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  
        
        
        metrics = compute_metrics(pred_flat, labels_flat)
        val_acc.append(metrics["accuracy"])
        val_prec.append(metrics["precision"])
        val_rec.append(metrics["recall"])
        val_f1.append(metrics["f1"])
#         nb_eval_steps += 1
        
#     print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
#     print(F'\n\tValidation MCC Accuracy: {eval_mcc_accuracy / nb_eval_steps}') # eval M
    
#     val_loss = val_loss.to('cpu').numpy()
    print(F'\n\tValidation loss: {np.mean(val_loss)}')
    print(F'\n\tValidation acc: {np.mean(val_acc)}')
    print(F'\n\tValidation prec: {np.mean(val_prec)}')
    print(F'\n\tValidation rec: {np.mean(val_rec)}')
    print(F'\n\tValidation f1: {np.mean(val_f1)}')


  0%|          | 0/83 [00:00<?, ?it/s]

<====================== Epoch 1 ======================>


<ipython-input-14-50b3d035d7dd>:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)


loss: 


  1%|          | 1/83 [00:22<30:16, 22.15s/it]

...

tensor(0.6761, device='cuda:0', grad_fn=<NllLossBackward>)



100%|██████████| 83/83 [37:35<00:00, 27.18s/it]

  0%|          | 0/42 [00:00<?, ?it/s]

loss: tensor(0.8894, device='cuda:0', grad_fn=<NllLossBackward>)

	Train loss: 0.6007264522184809

	train acc: 0.6543503559693319

	training prec: 0.8523426254377233

	training rec: 0.6543503559693319

	training f1: 0.7146045344378537

	Current Learning rate:  0.0



Epoch: 100%|██████████| 35/35 [19:44:31<00:00, 2030.62s/it]


	Validation loss: 0.5801786688112077

	Validation acc: 0.6202876984126984

	Validation prec: 0.8446590315195364

	Validation rec: 0.6202876984126984

	Validation f1: 0.6808587773163859


In [21]:
# train_loss.numpy()
# train_loss


In [22]:
# # Saving models
# torch.save(model.state_dict(), "finetuned.pth")

# #load models
# model = Neural

In [23]:
## saving the model 
torch.save(model.state_dict(), "finetuned-35-epochs-with-weighted-loss_lr_5e5.pth")

In [24]:
# loading the locally saved model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = CausalityBERT()
model.load_state_dict(torch.load("finetuned-35-epochs-with-weighted-loss_lr_5e5.pth"))
## Move the model to the GPU 
model.to(device)
model.eval() # gettign in the eval mode 



You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing BertModel: ['roberta.encoder.layer.0.output.dense.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.encoder.layer.9.output.dense.weight', 'roberta.encoder.layer.5.attention.self.key.weight', 'roberta.encoder.layer.7.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.7.intermediate.dense.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.attention.self.key.weight', 'lm_head.bias', 'roberta.encoder.layer.6.attention.self.query.bias', 'roberta.encoder.layer.11.attention.self.query.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.encoder.layer.4.intermediate.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'robe

CausalityBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True

## evaluation on the test dataset 



In [25]:
## evaluation on the test dataset 

# test_accuracy, test_mcc_accuracy, nb_test_steps = 0, 0, 0 
# test_mcc_accuracy, nb_test_steps = 0, 0,

# test_accuracy = []
# test_loss = []
# test_acc = []
# test_prec = []
# test_rec = []
# test_f1 = []

# for batch in tqdm(test_loader):
#     batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
#     b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader
#     with torch.no_grad():
#         model.eval()
#         logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 
    
    
#     loss = loss_fn(logits, b_labels)
#     test_loss.append(loss.item())

#     # move logits and labels to CPU
#     logits = logits.detach().to('cpu').numpy()
#     label_ids = b_labels.to('cpu').numpy()

#     pred_flat = np.argmax(logits, axis=1).flatten()
#     labels_flat = label_ids.flatten()

# #     eval_accuracy += accuracy_score(labels_flat, pred_flat)
# #     test_mcc_accuracy += matthews_corrcoef(labels_flat, pred_flat)  


#     metrics = compute_metrics(pred_flat, labels_flat)
#     test_acc.append(metrics["accuracy"])
#     test_prec.append(metrics["precision"])
#     test_rec.append(metrics["recall"])
#     test_f1.append(metrics["f1"])
# #     nb_test_steps += 1
    

# # print(F'\n\tValidation Accuracy: {eval_accuracy / nb_eval_steps}')
# # print(F'\n\ttest MCC Accuracy: {test_mcc_accuracy / nb_test_steps}') # eval M
# print(F'\n\ttest loss: {np.mean(test_loss)}')
# print(F'\n\ttest acc: {np.mean(test_acc)}')
# print(F'\n\ttest prec: {np.mean(test_prec)}')
# print(F'\n\ttest rec: {np.mean(test_rec)}')
# print(F'\n\ttest f1: {np.mean(test_f1)}')

In [26]:
loss_fn = CrossEntropyLoss()
test_loss = []
test_acc = []
test_prec = []
test_rec = []
test_f1 = []


for batch in tqdm(test_loader):
    batch = tuple(batch[t].to(device) for t in batch)      # batch to GPU
    b_input_ids, b_input_mask, b_token_type_ids, b_labels = batch     # unpack inputs from dataloader

    with torch.no_grad():
        model.eval()
        logits = model(**{"input_ids":b_input_ids, "attention_mask":b_input_mask, "token_type_ids":b_token_type_ids}) # forward pass, calculates logit predictions 
    
    print("logits:")
    print(logits)
    # move logits and labels to CPU
    logits = logits.detach().to('cpu').numpy()
    label_ids = b_labels.to('cpu').numpy()

    pred_flat = np.argmax(logits, axis=1).flatten()
    labels_flat = label_ids.flatten()
    print("pred:", pred_flat)
    print("real:", labels_flat)
    
    metrics = compute_metrics(pred_flat, labels_flat)
    test_acc.append(metrics["accuracy"])
    test_prec.append(metrics["precision"])
    test_rec.append(metrics["recall"])
    test_f1.append(metrics["f1"])

    
print(F'\n\ttest loss: {np.mean(test_loss)}')
print(F'\n\ttest acc: {np.mean(test_acc)}')
print(F'\n\ttest prec: {np.mean(test_prec)}')
print(F'\n\ttest rec: {np.mean(test_rec)}')
print(F'\n\ttest f1: {np.mean(test_f1)}')





  0%|          | 0/52 [00:00<?, ?it/s]<ipython-input-14-50b3d035d7dd>:17: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  , "labels" : torch.tensor(self.labels[idx], dtype=torch.long)
  2%|▏         | 1/52 [00:04<03:49,  4.51s/it]

logits:
tensor([[ 0.4904, -0.2755],
        [-0.1013,  0.4080],
        [ 0.3521, -0.2036],
        [-0.3749,  0.6805],
        [ 0.8254, -0.6578],
        [ 0.7953, -0.6137],
        [ 1.2399, -1.0747],
        [ 0.9515, -0.8847],
        [-0.1822,  0.5042],
        [ 0.1704,  0.0170],
        [ 0.7373, -0.5463],
        [-0.4168,  0.7147],
        [ 1.0940, -0.9389],
        [ 0.6103, -0.4381],
        [-0.4203,  0.7287],
        [ 1.1089, -0.9114],
        [ 0.1519,  0.1079],
        [ 0.0884,  0.1384],
        [ 0.8410, -0.6802],
        [-0.3653,  0.6362],
        [ 0.0813,  0.1463],
        [ 1.0591, -0.8933],
        [-0.3605,  0.6379],
        [-0.0293,  0.3212],
        [ 0.1037,  0.1493],
        [ 0.5919, -0.3467],
        [-0.3024,  0.5542],
        [ 0.0366,  0.2134],
        [ 0.7869, -0.5544],
        [ 0.0806,  0.1845],
        [ 0.3470, -0.1904],
        [-0.2136,  0.4390]], device='cuda:0')
pred: [0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1]
real: 

  4%|▍         | 2/52 [00:09<03:45,  4.51s/it]

logits:
tensor([[ 0.8242, -0.6847],
        [ 0.1357,  0.1073],
        [ 0.9094, -0.7099],
        [ 0.0940,  0.1681],
        [-0.0374,  0.3418],
        [ 1.0025, -0.8492],
        [ 0.8681, -0.6322],
        [ 0.8996, -0.7066],
        [ 0.5499, -0.3393],
        [-0.0636,  0.4098],
        [ 0.9420, -0.8431],
        [-0.4836,  0.7619],
        [ 0.1729,  0.0881],
        [-0.0044,  0.2906],
        [ 0.1902,  0.0671],
        [ 1.2415, -1.0608],
        [ 0.0604,  0.1817],
        [ 0.6077, -0.4325],
        [ 0.0063,  0.2267],
        [-0.1007,  0.3567],
        [ 0.8361, -0.6892],
        [ 0.7613, -0.6424],
        [ 0.3756, -0.2493],
        [ 0.0450,  0.2268],
        [-0.3300,  0.6435],
        [ 0.4876, -0.3139],
        [-0.4127,  0.7006],
        [ 0.9406, -0.7156],
        [ 0.1544,  0.0218],
        [-0.1705,  0.4815],
        [-0.1925,  0.5065],
        [-0.1105,  0.3976]], device='cuda:0')
pred: [0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 1]
real: 

  6%|▌         | 3/52 [00:13<03:38,  4.47s/it]

...

 98%|█████████▊| 51/52 [03:47<00:04,  4.47s/it]

logits:
tensor([[-0.1060,  0.4276],
        [ 0.2000, -0.0119],
        [ 0.5570, -0.4207],
        [-0.2242,  0.5221],
        [-0.0836,  0.3740],
        [-0.2122,  0.4744],
        [ 1.0564, -1.0196],
        [ 0.7444, -0.6304],
        [ 0.5539, -0.4055],
        [ 0.6517, -0.4901],
        [-0.0911,  0.3782],
        [ 0.5328, -0.3625],
        [ 0.3556, -0.2233],
        [-0.2936,  0.6367],
        [ 0.8998, -0.7104],
        [-0.2913,  0.5609],
        [-0.0990,  0.3981],
        [ 1.1645, -1.0131],
        [ 0.7612, -0.5847],
        [ 0.4804, -0.2382],
        [ 0.8777, -0.7947],
        [-0.2117,  0.5059],
        [ 0.3449, -0.0773],
        [ 0.2235,  0.0348],
        [-0.3917,  0.6808],
        [-0.1897,  0.5011],
        [ 1.2086, -1.0549],
        [-0.2604,  0.5550],
        [ 1.1848, -1.0110],
        [ 0.6442, -0.4583],
        [-0.3316,  0.6187],
        [ 0.2443, -0.0268]], device='cuda:0')
pred: [1 0 0 1 1 1 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0]
real: 

100%|██████████| 52/52 [03:49<00:00,  4.41s/it]

logits:
tensor([[-0.2203,  0.5152],
        [-0.3949,  0.6615],
        [ 0.8530, -0.6013],
        [ 0.6808, -0.4814],
        [ 0.8936, -0.6998],
        [ 0.0837,  0.1973],
        [ 0.2397, -0.0516],
        [-0.3773,  0.6604],
        [ 0.9102, -0.6717],
        [-0.2235,  0.5463],
        [-0.2756,  0.5374],
        [-0.0116,  0.2670],
        [ 0.6899, -0.5265],
        [-0.2293,  0.5114],
        [ 0.6151, -0.4156]], device='cuda:0')
pred: [1 1 0 0 0 1 0 1 0 1 1 1 0 1 0]
real: [0 1 0 0 0 1 0 0 0 0 0 0 0 1 0]

	test loss: nan

	test acc: 0.6131810897435898

	test prec: 0.8698094102077625

	test rec: 0.6131810897435898

	test f1: 0.6790997402485328



/home/vivek/miniconda3/envs/jmir/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/vivek/miniconda3/envs/jmir/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [27]:
print("pred_flat:", "\n", pred_flat)
print("\n")
print("logits", "\n", logits)

pred_flat: 
 [1 1 0 0 0 1 0 1 0 1 1 1 0 1 0]


logits 
 [[-0.2203012   0.51520926]
 [-0.39488778  0.6615116 ]
 [ 0.85299224 -0.6012564 ]
 [ 0.68075854 -0.48136777]
 [ 0.8935856  -0.69978046]
 [ 0.08367971  0.19733039]
 [ 0.23965524 -0.05155321]
 [-0.37729228  0.6603529 ]
 [ 0.9102003  -0.6717322 ]
 [-0.22349375  0.5463257 ]
 [-0.27564353  0.5373843 ]
 [-0.01159942  0.2670028 ]
 [ 0.6898548  -0.52651393]
 [-0.22931378  0.51137674]
 [ 0.61511284 -0.4155641 ]]


### printig some variabels to look into 

In [28]:
train_loss


[0.5875257253646851,
 0.6241099238395691,
 0.5964183807373047,
 0.4867846369743347,
 0.5664580464363098,
 0.6384475827217102,
 0.5234268307685852,
 0.611117422580719,
 0.5452214479446411,
 0.634705126285553,
 0.602626621723175,
 0.5830845832824707,
 0.5167552828788757,
 0.6032499074935913,
 0.6057895421981812,
 0.5802444815635681,
 0.6778051853179932,
 0.5698618292808533,
 0.6553929448127747,
 0.5920110940933228,
 0.5842699408531189,
 0.7046537399291992,
 0.617445170879364,
 0.5529775023460388,
 0.5285909175872803,
 0.6599240303039551,
 0.4906185269355774,
 0.5913090109825134,
 0.5554555654525757,
 0.6124575138092041,
 0.5329173803329468,
 0.6130095720291138,
 0.6332884430885315,
 0.5746018886566162,
 0.47806593775749207,
 0.70933997631073,
 0.5713217258453369,
 0.47686660289764404,
 0.7801812887191772,
 0.658084511756897,
 0.5817245244979858,
 0.6217321753501892,
 0.6044613718986511,
 0.6423258185386658,
 0.6924811601638794,
 0.6500186920166016,
 0.49954748153686523,
 0.63042032718658

In [29]:
train_acc 

[0.609375,
 0.625,
 0.65625,
 0.765625,
 0.625,
 0.609375,
 0.640625,
 0.625,
 0.703125,
 0.65625,
 0.765625,
 0.671875,
 0.6875,
 0.609375,
 0.640625,
 0.671875,
 0.609375,
 0.6875,
 0.640625,
 0.65625,
 0.640625,
 0.578125,
 0.625,
 0.65625,
 0.703125,
 0.640625,
 0.6875,
 0.65625,
 0.6875,
 0.71875,
 0.71875,
 0.625,
 0.703125,
 0.703125,
 0.703125,
 0.625,
 0.703125,
 0.734375,
 0.578125,
 0.59375,
 0.65625,
 0.65625,
 0.609375,
 0.640625,
 0.625,
 0.625,
 0.640625,
 0.59375,
 0.65625,
 0.796875,
 0.59375,
 0.640625,
 0.703125,
 0.6875,
 0.53125,
 0.578125,
 0.578125,
 0.609375,
 0.71875,
 0.734375,
 0.625,
 0.65625,
 0.75,
 0.640625,
 0.625,
 0.734375,
 0.6875,
 0.703125,
 0.65625,
 0.609375,
 0.625,
 0.640625,
 0.625,
 0.65625,
 0.59375,
 0.6875,
 0.546875,
 0.75,
 0.609375,
 0.734375,
 0.6875,
 0.609375,
 0.5454545454545454]

In [30]:
train_prec

[0.8091440886699507,
 0.7909919028340081,
 0.8870957051961823,
 0.9184782608695652,
 0.8299595141700405,
 0.825987539766702,
 0.8715992647058823,
 0.8442118226600985,
 0.9141301406926406,
 0.7913141025641026,
 0.8822841183574879,
 0.8886295995670995,
 0.8958333333333334,
 0.8508620689655172,
 0.7947115384615384,
 0.9017628205128205,
 0.7959577429149798,
 0.8546794871794872,
 0.885546875,
 0.9339541357370095,
 0.8082217261904762,
 0.7586649149149151,
 0.8209459459459459,
 0.866826923076923,
 0.924626245847176,
 0.7947115384615384,
 0.9189814814814815,
 0.8765624999999999,
 0.8546794871794872,
 0.9059794372294373,
 0.8722230913642053,
 0.7195945945945946,
 0.8363433441558442,
 0.8233072916666667,
 0.9595170454545454,
 0.8839743589743589,
 0.793359375,
 0.8783653846153846,
 0.7200755567338282,
 0.8692956349206349,
 0.866826923076923,
 0.765625,
 0.9191066066066066,
 0.8647836538461539,
 0.8125,
 0.8958333333333334,
 0.8702256944444444,
 0.8697916666666666,
 0.9004301619433198,
 0.84084821

In [31]:

train_rec

[0.609375,
 0.625,
 0.65625,
 0.765625,
 0.625,
 0.609375,
 0.640625,
 0.625,
 0.703125,
 0.65625,
 0.765625,
 0.671875,
 0.6875,
 0.609375,
 0.640625,
 0.671875,
 0.609375,
 0.6875,
 0.640625,
 0.65625,
 0.640625,
 0.578125,
 0.625,
 0.65625,
 0.703125,
 0.640625,
 0.6875,
 0.65625,
 0.6875,
 0.71875,
 0.71875,
 0.625,
 0.703125,
 0.703125,
 0.703125,
 0.625,
 0.703125,
 0.734375,
 0.578125,
 0.59375,
 0.65625,
 0.65625,
 0.609375,
 0.640625,
 0.625,
 0.625,
 0.640625,
 0.59375,
 0.65625,
 0.796875,
 0.59375,
 0.640625,
 0.703125,
 0.6875,
 0.53125,
 0.578125,
 0.578125,
 0.609375,
 0.71875,
 0.734375,
 0.625,
 0.65625,
 0.75,
 0.640625,
 0.625,
 0.734375,
 0.6875,
 0.703125,
 0.65625,
 0.609375,
 0.625,
 0.640625,
 0.625,
 0.65625,
 0.59375,
 0.6875,
 0.546875,
 0.75,
 0.609375,
 0.734375,
 0.6875,
 0.609375,
 0.5454545454545454]

In [32]:

train_f1

[0.6628313166234514,
 0.6757246376811595,
 0.7358035714285714,
 0.8042068506817426,
 0.6883604505632039,
 0.6903300243817485,
 0.6949816335350043,
 0.6820175438596492,
 0.7716011184451778,
 0.6969353864734299,
 0.8046723300970873,
 0.7475591309130913,
 0.7301136363636364,
 0.6751559251559253,
 0.688652073732719,
 0.7402934818756237,
 0.6685387864823349,
 0.7344336670838548,
 0.7238658133054684,
 0.7547605580693816,
 0.6846279890521463,
 0.6356363231363231,
 0.6820652173913044,
 0.720703125,
 0.7795631067961165,
 0.688652073732719,
 0.7461670838548186,
 0.7278061224489796,
 0.7344336670838548,
 0.7766071428571427,
 0.7795347341337907,
 0.6524086378737542,
 0.7454792983039574,
 0.7375096006144395,
 0.7885558252427185,
 0.7117346938775511,
 0.7293120730620731,
 0.769873271889401,
 0.6323564593301436,
 0.6776439299123904,
 0.720703125,
 0.6911231884057971,
 0.7079730628533422,
 0.7081439393939393,
 0.6763157894736842,
 0.7214285714285713,
 0.672654029966417,
 0.6982142857142857,
 0.7278645

In [33]:
val_accuracy

[]

In [34]:
val_loss

[0.702800452709198,
 0.5250018835067749,
 0.7090986371040344,
 0.6518169641494751,
 0.5457489490509033,
 0.5479598641395569,
 0.532227635383606,
 0.5785199403762817,
 0.761624276638031,
 0.4775333106517792,
 0.5654107928276062,
 0.5686037540435791,
 0.5682851076126099,
 0.5628700256347656,
 0.5394037961959839,
 0.46262815594673157,
 0.4998567998409271,
 0.5535305738449097,
 0.46309375762939453,
 0.4850519001483917,
 0.5246384739875793,
 0.6416539549827576,
 0.7478864192962646,
 0.6019914746284485,
 0.6380285620689392,
 0.54725581407547,
 0.6271235346794128,
 0.5508907437324524,
 0.5519490242004395,
 0.6776984930038452,
 0.5541132092475891,
 0.6372450590133667,
 0.664983332157135,
 0.5127569437026978,
 0.5362096428871155,
 0.8163884878158569,
 0.5023529529571533,
 0.537419855594635,
 0.48321136832237244,
 0.5684155821800232,
 0.5723525881767273,
 0.5718719959259033]

In [35]:
val_acc

[0.46875,
 0.71875,
 0.53125,
 0.5,
 0.6875,
 0.625,
 0.625,
 0.53125,
 0.46875,
 0.71875,
 0.65625,
 0.65625,
 0.65625,
 0.625,
 0.65625,
 0.75,
 0.6875,
 0.59375,
 0.6875,
 0.65625,
 0.6875,
 0.59375,
 0.59375,
 0.5625,
 0.5,
 0.65625,
 0.65625,
 0.65625,
 0.65625,
 0.625,
 0.625,
 0.5625,
 0.53125,
 0.71875,
 0.71875,
 0.5625,
 0.71875,
 0.625,
 0.65625,
 0.6875,
 0.625,
 0.3333333333333333]

In [36]:
val_prec

[1.0,
 0.8838068181818183,
 0.7887254901960784,
 0.8568627450980392,
 0.9479166666666666,
 0.8729757085020242,
 0.9464285714285714,
 0.9013157894736842,
 0.798828125,
 0.9134615384615384,
 0.8765624999999999,
 0.7984375,
 0.8272058823529411,
 0.8531746031746031,
 0.9471153846153846,
 0.9166666666666666,
 0.8958333333333334,
 0.689453125,
 0.9278846153846154,
 0.9263392857142857,
 0.8624999999999999,
 0.7669117647058824,
 0.7834821428571429,
 0.84375,
 0.8333333333333334,
 0.8272058823529411,
 0.8765624999999999,
 0.898538961038961,
 0.9713541666666666,
 0.6953125,
 0.925,
 0.8242647058823529,
 0.861328125,
 0.8995535714285714,
 0.8674242424242423,
 0.6972402597402597,
 0.9488636363636364,
 0.9464285714285714,
 0.9263392857142857,
 0.7491883116883117,
 0.7909919028340081,
 0.1111111111111111]

In [37]:
val_rec

[0.46875,
 0.71875,
 0.53125,
 0.5,
 0.6875,
 0.625,
 0.625,
 0.53125,
 0.46875,
 0.71875,
 0.65625,
 0.65625,
 0.65625,
 0.625,
 0.65625,
 0.75,
 0.6875,
 0.59375,
 0.6875,
 0.65625,
 0.6875,
 0.59375,
 0.59375,
 0.5625,
 0.5,
 0.65625,
 0.65625,
 0.65625,
 0.65625,
 0.625,
 0.625,
 0.5625,
 0.53125,
 0.71875,
 0.71875,
 0.5625,
 0.71875,
 0.625,
 0.65625,
 0.6875,
 0.625,
 0.3333333333333333]

In [38]:
val_f1

[0.6382978723404256,
 0.7751696832579186,
 0.6096491228070176,
 0.5954545454545455,
 0.7678571428571429,
 0.703125,
 0.7187499999999999,
 0.598356309650053,
 0.5737573099415204,
 0.7662703379224031,
 0.7278061224489796,
 0.7014236545682103,
 0.693936877076412,
 0.6884057971014493,
 0.7437074829931973,
 0.7916666666666666,
 0.7301136363636364,
 0.6141666666666667,
 0.7526041666666666,
 0.7272371714643305,
 0.7395833333333334,
 0.6382890365448505,
 0.6493421052631578,
 0.634090909090909,
 0.5757575757575758,
 0.693936877076412,
 0.7278061224489796,
 0.7449095022624433,
 0.7646116138763197,
 0.6386639676113361,
 0.7010869565217391,
 0.6193181818181819,
 0.6239035087719299,
 0.7572368421052632,
 0.7642857142857143,
 0.6171875,
 0.791289592760181,
 0.7187499999999999,
 0.7272371714643305,
 0.7086352657004831,
 0.6757246376811595,
 0.16666666666666666]

In [39]:
# test_accuracy

In [40]:
test_loss

[]

In [41]:
test_acc

[0.625,
 0.71875,
 0.65625,
 0.59375,
 0.78125,
 0.625,
 0.5625,
 0.5625,
 0.625,
 0.34375,
 0.53125,
 0.71875,
 0.71875,
 0.5,
 0.625,
 0.65625,
 0.46875,
 0.5625,
 0.59375,
 0.53125,
 0.5,
 0.59375,
 0.6875,
 0.59375,
 0.625,
 0.5625,
 0.5625,
 0.65625,
 0.59375,
 0.71875,
 0.71875,
 0.625,
 0.71875,
 0.5,
 0.625,
 0.6875,
 0.71875,
 0.5625,
 0.65625,
 0.6875,
 0.5,
 0.5625,
 0.5625,
 0.53125,
 0.59375,
 0.59375,
 0.71875,
 0.71875,
 0.5625,
 0.625,
 0.65625,
 0.6666666666666666]

In [42]:
test_prec

[0.925,
 0.8995535714285714,
 0.9263392857142857,
 0.8485294117647059,
 0.8251811594202898,
 0.7909919028340081,
 0.9708333333333333,
 0.84375,
 0.8618421052631579,
 0.7598958333333333,
 0.970703125,
 0.8838068181818183,
 0.9488636363636364,
 0.8809523809523809,
 0.8531746031746031,
 0.898538961038961,
 0.8519345238095238,
 0.7757352941176471,
 0.8485294117647059,
 0.861328125,
 0.7916666666666666,
 0.751953125,
 0.9278846153846154,
 0.7834821428571429,
 0.90625,
 0.8898809523809523,
 0.9227941176470589,
 0.8577935222672064,
 0.8021255060728746,
 0.8418522267206477,
 0.9296875,
 0.8531746031746031,
 0.8536458333333333,
 0.8828125,
 0.90625,
 0.847419028340081,
 0.9488636363636364,
 0.9027777777777778,
 0.9263392857142857,
 0.8611111111111112,
 0.8568627450980392,
 0.9227941176470589,
 0.9227941176470589,
 0.9013157894736842,
 0.8021255060728746,
 0.8021255060728746,
 0.8674242424242423,
 0.8875,
 0.9453125,
 0.7757936507936508,
 0.8577935222672064,
 0.875]

In [43]:
test_rec

[0.625,
 0.71875,
 0.65625,
 0.59375,
 0.78125,
 0.625,
 0.5625,
 0.5625,
 0.625,
 0.34375,
 0.53125,
 0.71875,
 0.71875,
 0.5,
 0.625,
 0.65625,
 0.46875,
 0.5625,
 0.59375,
 0.53125,
 0.5,
 0.59375,
 0.6875,
 0.59375,
 0.625,
 0.5625,
 0.5625,
 0.65625,
 0.59375,
 0.71875,
 0.71875,
 0.625,
 0.71875,
 0.5,
 0.625,
 0.6875,
 0.71875,
 0.5625,
 0.65625,
 0.6875,
 0.5,
 0.5625,
 0.5625,
 0.53125,
 0.59375,
 0.59375,
 0.71875,
 0.71875,
 0.5625,
 0.625,
 0.65625,
 0.6666666666666666]

In [44]:
test_f1

[0.7010869565217391,
 0.7572368421052632,
 0.7272371714643305,
 0.6616959064327486,
 0.7964285714285715,
 0.6757246376811595,
 0.6901041666666666,
 0.634090909090909,
 0.6523279352226721,
 0.4502253446447509,
 0.6632509386733416,
 0.7751696832579186,
 0.791289592760181,
 0.5485829959514169,
 0.6884057971014493,
 0.7449095022624433,
 0.5658222591362128,
 0.6221590909090909,
 0.6616959064327486,
 0.6239035087719299,
 0.5425101214574899,
 0.6286452810180276,
 0.7526041666666666,
 0.6493421052631578,
 0.6863636363636364,
 0.671875,
 0.6460227272727272,
 0.7143304130162703,
 0.6623904881101376,
 0.7486842105263157,
 0.7772959183673469,
 0.6884057971014493,
 0.7557102628285357,
 0.6183574879227053,
 0.6863636363636364,
 0.7297705314009661,
 0.791289592760181,
 0.6287878787878787,
 0.7272371714643305,
 0.7064777327935222,
 0.5954545454545455,
 0.6460227272727272,
 0.6460227272727272,
 0.598356309650053,
 0.6623904881101376,
 0.6623904881101376,
 0.7642857142857143,
 0.7495847176079735,
 0.666